In [1]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeRegressor
import torch
import torch.nn as nn

In [2]:
tr = pd.read_csv('price_data_tr.csv')

In [3]:
tr.describe()

,id,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
count,1.296800e+04,1.296800e+04,12968.000000,12968.000000,12968.000000,1.296800e+04,12968.000000,12968.000000,12968.000000,12968.000000,12968.000000,12968.000000,12968.000000,12968.000000,12968.000000,12968.000000,12968.000000,12968.000000,12968.000000,12968.000000
mean,4.571301e+09,5.387445e+05,3.369525,2.117829,2078.315623,1.495213e+04,1.497494,0.007711,0.233189,3.403763,7.658853,1786.903378,291.412246,1971.137878,85.090839,98078.229488,47.560215,-122.214346,1986.085287,12828.123535
std,2.879111e+09,3.619882e+05,0.904955,0.769985,916.026315,4.158912e+04,0.538748,0.087478,0.766119,0.640519,1.170140,824.992857,439.130219,29.258371,403.203649,53.524760,0.138838,0.141241,682.604738,28107.671504
min,1.000102e+06,8.000000e+04,0.000000,0.000000,370.000000,5.720000e+02,1.000000,0.000000,0.000000,1.000000,3.000000,370.000000,0.000000,1900.000000,0.000000,98001.000000,47.155900,-122.515000,460.000000,659.000000
25%,2.112701e+09,3.249500e+05,3.000000,1.750000,1421.500000,5.040000e+03,1.000000,0.000000,0.000000,3.000000,7.000000,1190.000000,0.000000,1952.000000,0.000000,98033.000000,47.470975,-122.329000,1490.000000,5100.000000
50%,3.905030e+09,4.500000e+05,3.000000,2.250000,1910.000000,7.609000e+03,1.500000,0.000000,0.000000,3.000000,7.000000,1560.000000,0.000000,1975.000000,0.000000,98065.000000,47.572950,-122.231000,1840.000000,7625.500000
75%,7.312125e+09,6.400000e+05,4.000000,2.500000,2540.000000,1.058725e+04,2.000000,0.000000,0.000000,4.000000,8.000000,2210.000000,550.000000,1997.000000,0.000000,98118.000000,47.678200,-122.125000,2360.000000,10053.500000
max,9.895000e+09,7.062500e+06,10.000000,8.000000,13540.000000,1.651359e+06,3.500000,1.000000,4.000000,5.000000,13.000000,9410.000000,4130.000000,2015.000000,2015.000000,98199.000000,47.777600,-121.315000,6110.000000,871200.000000


In [4]:
tr.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12968 entries, 0 to 12967
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             12968 non-null  int64  
 1   date           12968 non-null  object 
 2   price          12968 non-null  float64
 3   bedrooms       12968 non-null  int64  
 4   bathrooms      12968 non-null  float64
 5   sqft_living    12968 non-null  int64  
 6   sqft_lot       12968 non-null  int64  
 7   floors         12968 non-null  float64
 8   waterfront     12968 non-null  int64  
 9   view           12968 non-null  int64  
 10  condition      12968 non-null  int64  
 11  grade          12968 non-null  int64  
 12  sqft_above     12968 non-null  int64  
 13  sqft_basement  12968 non-null  int64  
 14  yr_built       12968 non-null  int64  
 15  yr_renovated   12968 non-null  int64  
 16  zipcode        12968 non-null  int64  
 17  lat            12968 non-null  float64
 18  long  

In [5]:
tr['long'].max(), tr['long'].min(), 

(-121.315, -122.515)

In [6]:
#tr['long'] = (tr['long']-tr['long'].mean())/(tr['long'].max() - tr['long'].min())

In [7]:
#finding NAN columns

nan_col = np.any(pd.isna(tr), axis = 0)
nan_col = list(nan_col[nan_col == True].index)

In [8]:
tr.columns

Index(['id', 'date', 'price', 'bedrooms', 'bathrooms', 'sqft_living',
       'sqft_lot', 'floors', 'waterfront', 'view', 'condition', 'grade',
       'sqft_above', 'sqft_basement', 'yr_built', 'yr_renovated', 'zipcode',
       'lat', 'long', 'sqft_living15', 'sqft_lot15'],
      dtype='object')

In [9]:
tr=tr.drop(['id', 'date'], axis=1)

In [10]:
tr.columns

Index(['price', 'bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot', 'floors',
       'waterfront', 'view', 'condition', 'grade', 'sqft_above',
       'sqft_basement', 'yr_built', 'yr_renovated', 'zipcode', 'lat', 'long',
       'sqft_living15', 'sqft_lot15'],
      dtype='object')

In [11]:
tr_x_col = list(tr.columns)   # make a list of columns
tr_x_col.remove('price')
tr_y_col = ['price']

tr_x_col, tr_y_col

(['bedrooms',
  'bathrooms',
  'sqft_living',
  'sqft_lot',
  'floors',
  'waterfront',
  'view',
  'condition',
  'grade',
  'sqft_above',
  'sqft_basement',
  'yr_built',
  'yr_renovated',
  'zipcode',
  'lat',
  'long',
  'sqft_living15',
  'sqft_lot15'],
 ['price'])

In [12]:
tr_x_df = pd.DataFrame(tr, columns=tr_x_col)
tr_y_df = pd.DataFrame(tr, columns=tr_y_col)

In [13]:
tr_x = torch.tensor(tr_x_df.values, dtype=torch.float)
tr_y = torch.tensor(tr_y_df.values, dtype=torch.float)

In [14]:
tr_x.shape, tr_y.shape

(torch.Size([12968, 18]), torch.Size([12968, 1]))

In [15]:
#loading validation set
val = pd.read_csv('price_data_val.csv')

In [16]:
val = val.drop(['id', 'date'], axis=1)

val_x_col = list(val.columns)   # make a list of columns
val_x_col.remove('price')
val_y_col = ['price']

val_x_col, val_y_col

(['bedrooms',
  'bathrooms',
  'sqft_living',
  'sqft_lot',
  'floors',
  'waterfront',
  'view',
  'condition',
  'grade',
  'sqft_above',
  'sqft_basement',
  'yr_built',
  'yr_renovated',
  'zipcode',
  'lat',
  'long',
  'sqft_living15',
  'sqft_lot15'],
 ['price'])

In [17]:
val_x_df = pd.DataFrame(val, columns=val_x_col)
val_y_df = pd.DataFrame(val, columns=val_y_col)

val_x = torch.tensor(val_x_df.values, dtype=torch.float)
val_y = torch.tensor(val_y_df.values, dtype=torch.float)

val_x.shape, val_y.shape

(torch.Size([4323, 18]), torch.Size([4323, 1]))

In [18]:
dtr = DecisionTreeRegressor().fit(tr_x, tr_y)
print(dtr.score(tr_x, tr_y))

0.999495665383272


In [19]:
res = dtr.predict(val_x)

In [20]:
res

array([342500., 509950., 400000., ..., 465000., 745000., 450000.])

In [21]:
type(val_y)

torch.Tensor

In [22]:
val_y[2] == res[2]

tensor([False])

In [23]:
#val_y = val_y.numpy()
count = 0
for i in range(len(res)):
    if (res[i] == val_y[i]):
        count += 1
        print(res[i])
print(count)

500000.0
317000.0
310000.0
635000.0
585000.0
539950.0
430000.0
950000.0
490000.0
375000.0
430000.0
575000.0
437500.0
450000.0
255000.0
375000.0
392000.0
330000.0
925000.0
870000.0
940000.0
340000.0
210000.0
540000.0
610000.0
850000.0
350000.0
799950.0
990000.0
420000.0
525000.0
255000.0
685000.0
305000.0
415000.0
428000.0
350000.0
700000.0
38


In [24]:
# norma;izing values

tr_x = (tr_x - tr_x.mean())/ tr_x.std()
val_x = (val_x - val_x.mean())/ val_x.std()

In [25]:
dtr = DecisionTreeRegressor().fit(tr_x, tr_y)
print(dtr.score(tr_x, tr_y))

0.9994929630131416


In [26]:
res2 = dtr.predict(val_x)

In [27]:
#val_y_np = val_y.numpy()
count = 0
for i in range(len(res)):
    if (res[i] == val_y[i]):
        count += 1
        print(res[i])
print(count)

500000.0
317000.0
310000.0
635000.0
585000.0
539950.0
430000.0
950000.0
490000.0
375000.0
430000.0
575000.0
437500.0
450000.0
255000.0
375000.0
392000.0
330000.0
925000.0
870000.0
940000.0
340000.0
210000.0
540000.0
610000.0
850000.0
350000.0
799950.0
990000.0
420000.0
525000.0
255000.0
685000.0
305000.0
415000.0
428000.0
350000.0
700000.0
38


In [28]:
test = pd.read_csv('price_data_ts.csv')
test = test.drop(['id', 'date'], axis=1)

test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4322 entries, 0 to 4321
Data columns (total 19 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   price          0 non-null      float64
 1   bedrooms       4322 non-null   int64  
 2   bathrooms      4322 non-null   float64
 3   sqft_living    4322 non-null   int64  
 4   sqft_lot       4322 non-null   int64  
 5   floors         4322 non-null   float64
 6   waterfront     4322 non-null   int64  
 7   view           4322 non-null   int64  
 8   condition      4322 non-null   int64  
 9   grade          4322 non-null   int64  
 10  sqft_above     4322 non-null   int64  
 11  sqft_basement  4322 non-null   int64  
 12  yr_built       4322 non-null   int64  
 13  yr_renovated   4322 non-null   int64  
 14  zipcode        4322 non-null   int64  
 15  lat            4322 non-null   float64
 16  long           4322 non-null   float64
 17  sqft_living15  4322 non-null   int64  
 18  sqft_lot

In [29]:
val_x_col
test_x_df = pd.DataFrame(test, columns=val_x_col)
test_x = torch.tensor(test_x_df.values, dtype=torch.float)

In [30]:
res2 = dtr.predict(test_x)
np.unique(res2)

array([2280000.])

In [31]:
test = pd.read_csv('price_data_ts.csv')
res_col = ['id', 'price']
res_df = pd.DataFrame(test, columns=res_col)
res_df['id'], res_df['price'] = test['id'], res2

In [32]:
res_df.to_csv('result.csv')

In [33]:
res_df['id'] = res_df['id'].astype(str)+test['date'].astype(str)

In [34]:
#addinf 0 in the front
res_df['id'] = str(0)+res_df['id'].astype(str)
res_df['id']

0       0700010085020140926T000000
1       0403740028020140923T000000
2       0142340016020140618T000000
3       0776740006020141119T000000
4       0946590050020140617T000000
                   ...            
4317    0733822037020141006T000000
4318    0681910015020140721T000000
4319     082405914020140527T000000
4320     051450009020140513T000000
4321    0770180005020140625T000000
Name: id, Length: 4322, dtype: object

In [35]:
res_df.to_csv('result.csv', index=False)

In [36]:
res_df

,id,price
0,0700010085020140926T000000,2280000.0
1,0403740028020140923T000000,2280000.0
2,0142340016020140618T000000,2280000.0
3,0776740006020141119T000000,2280000.0
4,0946590050020140617T000000,2280000.0
...,...,...
4317,0733822037020141006T000000,2280000.0
4318,0681910015020140721T000000,2280000.0
4319,082405914020140527T000000,2280000.0
4320,051450009020140513T000000,2280000.0
